In [ ]:
import json
import shutil
from pathlib import Path

In [ ]:
tree = Path("..")

In [ ]:
delete_me = [".ipynb_checkpoints", "__pycache__", ".virtual_documents"]


def CleanTreeOfTrash(tree):
    for item in tree.iterdir():
        if item.is_dir():
            if any([item.name == s for s in delete_me]):
                print("Removing", item)
                try:
                    shutil.rmtree(str(item))
                except Exception:
                    pass
            else:
                CleanTreeOfTrash(item)


CleanTreeOfTrash(tree)

In [ ]:
from collections import defaultdict

skip_folders = [".git"]


class Result:
    def __init__(self):
        self.keys = []

    def __repr__(self):
        return ", ".join(self.keys)


def IterTree(tree):
    discovered = defaultdict(Result)
    for item in tree.iterdir():
        if item.is_dir():
            if item.name in skip_folders:
                continue
            discovered.update(IterTree(item))
        else:
            try:
                data = item.read_text().lower()
                for search in [
                    "pw",
                    "password",
                    "token",
                    "cred",
                    "auth",
                    "bearer",
                    "user",
                    "client_secret",
                ]:
                    if search in data:
                        discovered[item].keys.append(search)
            except UnicodeDecodeError:
                pass
            except Exception as e:
                print(type(e), e)
                print()
    return dict(discovered)


results = IterTree(tree)
results

In [ ]:
def SecureIPythonOutput(tree):
    for item in tree.iterdir():
        if item.is_dir():
            SecureIPythonOutput(item)
        else:
            if item.suffix == ".ipynb":
                print(f"cleaning {item}")
                data = json.loads(item.read_text())
                for cell in data["cells"]:
                    if cell["cell_type"] == "code":
                        if len(cell["outputs"]) > 0:
                            print(f'\tremoving {len(cell["outputs"])} lines of output')
                            cell["outputs"] = []
                item.write_text(json.dumps(data, indent=4))


SecureIPythonOutput(tree)